In [49]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv

def get_data(section_name, file_name):
    page = 1
    stop = False
    cutoff_date = datetime.strptime("01.01.2020", "%d.%m.%Y").date()
    
    print(f"Раздел - {section_name}")

    with open(file_name, mode="w", newline="",encoding="utf8") as file:
        writer = csv.writer(file)
        writer.writerow(["date", "title", "description"])

        while not stop:
            print(f"Обработка страницы {page}...", end="\r")
            
            response = requests.get(f"https://kapital.kz/{section_name}?page={page}")
            if response.status_code != 200:
                print(f"Ошибка {response.status_code}")

            html_content = response.text
            soup = BeautifulSoup(html_content, "html.parser")

            articles = soup.find_all("article")
            for article in articles:
                date_str = article.find("time")["datetime"]
                date_obj = datetime.fromisoformat(date_str).date()
                if(date_obj < cutoff_date) :
                    print("Все данные получены!")
                    stop = True
                    break
                title = article.find("h3").text
                p_tag = article.find("p")
                if(not p_tag) : continue
                description = p_tag.text.strip()
                writer.writerow([date_obj.strftime("%d.%m.%Y"),title,description])
            page += 1

Получаем заголовки и короткое описание статей по разделам

In [50]:
sections = {
    "economic":"news_economic.csv",
    "finance":"news_finance.csv",
    "gosudarstvo":"news_gosudarstvo.csv",
    "tehnology":"news_tehnology.csv",
    "world":"news_world.csv",
    "business":"news_business.csv"
}
for section_name, file_name in sections.items():
    get_data(section_name, file_name)

Раздел - economic
Все данные получены!349...
Раздел - finance
Все данные получены!240...
Раздел - gosudarstvo
Все данные получены!481...
Раздел - tehnology
Все данные получены!42...
Раздел - world
Все данные получены!69...
Раздел - business
Все данные получены!67...


Раздел немного отличается вносим правки в парсер

In [61]:
page = 1
stop = False
cutoff_date = datetime.strptime("01.01.2020", "%d.%m.%Y").date()
    
with open("news_investments.csv", mode="w", newline="",encoding="utf8") as file:
    writer = csv.writer(file)
    writer.writerow(["date", "title", "description"])

    while not stop:
        print(f"Обработка страницы {page}...", end="\r")
            
        response = requests.get(f"https://kapital.kz/project/investments?page={page}")
        if response.status_code != 200:
            print(f"Ошибка {response.status_code}")

        html_content = response.text
        soup = BeautifulSoup(html_content, "html.parser")

        articles = soup.find_all("article")
        if not articles:
            print("Статьи кончились")
            stop = True
            break
        for article in articles:
            date_str = article.find("time")["datetime"]
            date_obj = datetime.fromisoformat(date_str).date()
            if(date_obj < cutoff_date) :
                print("Все данные получены!")
                stop = True
                break
            try:
                title = article.find_all("a")[2].text
            except:
                title = article.find_all("a")[1].text
            p_tag = article.find("p")
            if(not p_tag) : continue
            description = p_tag.text.strip()
            writer.writerow([date_obj.strftime("%d.%m.%Y"),title,description])
        page += 1

Статьи кончилисьцы 102...
